# Capstone Project

Pehumayen is a store that sell crafts made with wood in Santiago, specifically in Peñalolen district. due the good results, the owner is planning to open a new store and now he is evaluating where to put it. 

At this moment he is evaluating possible places, for the owner of peumayen it is of vital importance to position himself in a place where people have high purchasing power, since his craft is very expensive.

Therefore, the problem to solve is to locate possible places where Peumayen can open a new store, places that meet conditions similar to the place where it is currently ubicated, in Peñalolen.

Libreries

In [1]:
import requests
import csv
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import geocoder # import geocoder
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Get and process data
Import dataset with longitudes and latitudes (wep-scrapping)

In [2]:
source = requests.get('https://carta-natal.es/ciudades/Chile/Region_Metropolitana').text
sources=[]
data=pd.DataFrame()
for i in range(6):
    if i==0:
        source = requests.get('https://carta-natal.es/ciudades/Chile/Region_Metropolitana').text
        soup = BeautifulSoup(source, 'lxml')
        My_table = soup.table
        sources.append(source)
    else:
        source = requests.get('https://carta-natal.es/ciudades/Chile/Region_Metropolitana-'+str(i+1)).text
        sources.append(source)
        soup = BeautifulSoup(source, 'lxml')
        My_table = soup.table
    df_import = pd.read_html(str(My_table),header=0)[0]
    data=data.append(df_import)


In [3]:
data.reset_index(inplace=True,drop=True)
metrop=data
metrop

,Ciudad,Latitud,Longitud,Zona horaria
0,Alfalfalito,34°05′00″S,70°07′00″W,UT-3:00
1,Alicia de Penaflor,33°37′00″S,70°54′00″W,UT-3:00
2,Alto de Jahuel,33°44′00″S,70°42′00″W,UT-3:00
3,Angostura,33°54′00″S,70°44′00″W,UT-3:00
4,Apoquindo,33°24′00″S,70°32′00″W,UT-3:00
5,Asiento Viejo,33°04′00″S,70°59′00″W,UT-3:00
6,Bajos de Mena,33°38′00″S,70°37′00″W,UT-3:00
7,Banos de Colina,33°12′00″S,70°38′00″W,UT-3:00
8,Banos del Tupungato,33°24′00″S,69°59′00″W,UT-3:00
9,Barranca de Piche,33°59′00″S,71°04′00″W,UT-3:00


Change scale from DMS (degrees minutes and seconds to degrees)

In [4]:
data['A'], data['B'] = data['Latitud'].str.split('°', 1).str
data['C'], data['D'] = data['B'].str.split('′', 1).str
data['E'], data['F'] = data['D'].str.split('″', 1).str
data['lat']=(data['A'].astype(np.int64) + data['C'].astype(np.int64)/60 +data['E'].astype(np.int64)/3600)*-1

In [5]:
data['A'], data['B'] = data['Longitud'].str.split('°', 1).str
data['C'], data['D'] = data['B'].str.split('′', 1).str
data['E'], data['F'] = data['D'].str.split('″', 1).str
data['long']=(data['A'].astype(np.int64) + data['C'].astype(np.int64)/60 +data['E'].astype(np.int64)/3600)*-1

In [6]:
metrop['lat']=data['lat']
metrop['long']=data['long']
df=metrop

In [7]:
metrop

,Ciudad,Latitud,Longitud,Zona horaria,A,B,C,D,E,F,lat,long
0,Alfalfalito,34°05′00″S,70°07′00″W,UT-3:00,70,07′00″W,07,00″W,00,W,-34.083333,-70.116667
1,Alicia de Penaflor,33°37′00″S,70°54′00″W,UT-3:00,70,54′00″W,54,00″W,00,W,-33.616667,-70.900000
2,Alto de Jahuel,33°44′00″S,70°42′00″W,UT-3:00,70,42′00″W,42,00″W,00,W,-33.733333,-70.700000
3,Angostura,33°54′00″S,70°44′00″W,UT-3:00,70,44′00″W,44,00″W,00,W,-33.900000,-70.733333
4,Apoquindo,33°24′00″S,70°32′00″W,UT-3:00,70,32′00″W,32,00″W,00,W,-33.400000,-70.533333
5,Asiento Viejo,33°04′00″S,70°59′00″W,UT-3:00,70,59′00″W,59,00″W,00,W,-33.066667,-70.983333
6,Bajos de Mena,33°38′00″S,70°37′00″W,UT-3:00,70,37′00″W,37,00″W,00,W,-33.633333,-70.616667
7,Banos de Colina,33°12′00″S,70°38′00″W,UT-3:00,70,38′00″W,38,00″W,00,W,-33.200000,-70.633333
8,Banos del Tupungato,33°24′00″S,69°59′00″W,UT-3:00,69,59′00″W,59,00″W,00,W,-33.400000,-69.983333
9,Barranca de Piche,33°59′00″S,71°04′00″W,UT-3:00,71,04′00″W,04,00″W,00,W,-33.983333,-71.066667


Coordinates of Santiago

In [8]:
address = 'Santiago, Chile'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Santiago are {}, {}.'.format(latitude, longitude))

C:\Users\FelipeBahamonde\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Santiago are -33.4377968, -70.6504451.


In [9]:
# create map of New York using latitude and longitude values
map_santiago = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng,comuna in zip(df['lat'], df['long'], df['Ciudad']):
    label = '{}, {}, {}'.format(comuna,lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_santiago)  
    
map_santiago

The Euclidean distance to the center of Santiago is calculated to reduce the points to be used

In [10]:
df=df[np.sqrt((df['lat']-latitude)*(df['lat']-latitude)+(df['long']-longitude)*(df['long']-longitude))<0.2]
df

,Ciudad,Latitud,Longitud,Zona horaria,A,B,C,D,E,F,lat,long
4,Apoquindo,33°24′00″S,70°32′00″W,UT-3:00,70,32′00″W,32,00″W,00,W,-33.400000,-70.533333
6,Bajos de Mena,33°38′00″S,70°37′00″W,UT-3:00,70,37′00″W,37,00″W,00,W,-33.633333,-70.616667
10,Barrancas,33°27′00″S,70°46′00″W,UT-3:00,70,46′00″W,46,00″W,00,W,-33.450000,-70.766667
11,Barrio Bellavista,33°25′58″S,70°38′09″W,UT-3:00,70,38′09″W,38,09″W,09,W,-33.432778,-70.635833
12,Barrio Brazil,33°26′18″S,70°39′59″W,UT-3:00,70,39′59″W,39,59″W,59,W,-33.438333,-70.666389
20,Carrascal,33°25′00″S,70°43′00″W,UT-3:00,70,43′00″W,43,00″W,00,W,-33.416667,-70.716667
26,Chacarilla de Macul,33°30′00″S,70°35′00″W,UT-3:00,70,35′00″W,35,00″W,00,W,-33.500000,-70.583333
27,Chacra Valparaiso,33°28′00″S,70°36′00″W,UT-3:00,70,36′00″W,36,00″W,00,W,-33.466667,-70.600000
28,Chicureo Abajo,33°17′00″S,70°38′00″W,UT-3:00,70,38′00″W,38,00″W,00,W,-33.283333,-70.633333
35,Conchali,33°22′41″S,70°40′29″W,UT-3:00,70,40′29″W,40,29″W,29,W,-33.378056,-70.674722


In [13]:
# create map of New York using latitude and longitude values
map_santiago = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng,comuna in zip(df['lat'], df['long'], df['Ciudad']):
    label = '{}, {}, {}'.format(comuna,lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_santiago)  
    
map_santiago

## Foursquare API

In [12]:
CLIENT_ID = 'HJ3FF2KWWRKUN1VDFIUAXANKVXGT2BAOCJIWHHYRLX10QI3B' # your Foursquare ID
CLIENT_SECRET = 'GZGFX5AZZ1UOKRHZAQHZ42O3T1EKK1FZ5FNQYWY0TRMHBWOR' # your Foursquare Secret
VERSION = '20180927'

example with one district

In [14]:
df.loc[259, 'Ciudad']

'Santiago de Chile'

In [15]:
neighborhood_latitude = df.loc[259, 'lat'] # neighborhood latitude value
neighborhood_longitude = df.loc[259, 'long'] # neighborhood longitude value

neighborhood_name = df.loc[259, 'Ciudad'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Santiago de Chile are -33.456944444444446, -70.64833333333334.


In [16]:
# type your answer here
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=HJ3FF2KWWRKUN1VDFIUAXANKVXGT2BAOCJIWHHYRLX10QI3B&client_secret=GZGFX5AZZ1UOKRHZAQHZ42O3T1EKK1FZ5FNQYWY0TRMHBWOR&v=20180927&ll=-33.456944444444446,-70.64833333333334&radius=1500&limit=200'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb1976e4434b97357ff0ca3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b85a2ecf964a5201e6a31e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f1931735',
         'name': 'General Entertainment',
         'pluralName': 'General Entertainment',
         'primary': True,
         'shortName': 'Entertainment'}],
       'id': '4b85a2ecf964a5201e6a31e3',
       'location': {'address': 'San Diego 850',
        'cc': 'CL',
        'city': 'Santiago de Chile',
        'country': 'Chile',
        'crossStreet': 'Copiapó',
        'distance': 134,
        'formattedAddress': ['San Diego 850 (Copiapó)',
         '8330840 Santiago de Chile',
         

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Teatro Caupolicán,General Entertainment,-33.456184,-70.649455
1,Vg! Burguer,Vegetarian / Vegan Restaurant,-33.455889,-70.651211
2,Plaza Sacramentinos,Plaza,-33.451312,-70.648375
3,Juegos Diana,Theme Park Ride / Attraction,-33.451098,-70.650280
4,Teatro Diana,Theater,-33.450842,-70.648963


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# type your answer here
Santiago_venues = getNearbyVenues(names=df['Ciudad'],
                                   latitudes=df['lat'],
                                   longitudes=df['long']
                                  )


Apoquindo
Bajos de Mena
Barrancas
Barrio Bellavista
Barrio Brazil
Carrascal
Chacarilla de Macul
Chacra Valparaiso
Chicureo Abajo
Conchali
Conchali
El Almendro
El Blanqueado
El Bosque
El Bosque
El Guanaco
El Mirador
El Peral
El Peumo
El Retamo
El Rosal
Errazuriz
Espejo
Independencia
La Africana
La Cisterna
La Dehesa
La Dehesa
La Estrella
La Feria
La Florida
La Granja
La Lata
La Lata
La Legua de Macul
La Pintana
La Reina
Las Condes
Las Condes
Lastarria
Lo Amor
Lo Aranguiz
Lo Barnechea
Lo Canas
Lo Chena
Lo Contador
Lo Curro
Lo Hermida Abajo
Lo Lillo
Lo Ovalle
Lo Prado
Lo Valdivieso
Lo Vasquez
Los Cerrillos
Los Nogales
Los Quillayes
Macul
Macul
Maipu
Mapuhue
Monserrat
Nunoa
Ochagavia
Penalolen
Providencia
Puente Alto
Quilicura
Quinta Normal
Recoleta
Recoleta
Renca
Rinconada de Chena
San Bernardo
San Joaquin
San Joaquin
San Miguel
San Ramon
Santa Ana de Chena
Santa Carolina
Santa Julia
Santa Margarita
Santa Teresa de Lo Ovalle
Santiago
Santiago de Chile
Villa Presidente Frei, Nunoa, Santiag

In [23]:
print(Santiago_venues.shape)
Santiago_venues.head()

(3663, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Apoquindo,-33.4,-70.533333,Estadio San Jorge,-33.391910,-70.536040,Convention Center
1,Apoquindo,-33.4,-70.533333,Pueblito de los Domínicos,-33.408394,-70.541394,Arts & Crafts Store
2,Apoquindo,-33.4,-70.533333,Tenis El Alba,-33.404797,-70.536641,Tennis Court
3,Apoquindo,-33.4,-70.533333,Plaza Dunkerque,-33.394187,-70.541335,Plaza
4,Apoquindo,-33.4,-70.533333,Nueva China Premium,-33.393786,-70.545035,Chinese Restaurant


In [24]:
Santiago_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Apoquindo,40,40,40,40,40,40
Bajos de Mena,6,6,6,6,6,6
Barrancas,23,23,23,23,23,23
Barrio Bellavista,100,100,100,100,100,100
Barrio Brazil,100,100,100,100,100,100
Carrascal,21,21,21,21,21,21
Chacarilla de Macul,66,66,66,66,66,66
Chacra Valparaiso,100,100,100,100,100,100
Chicureo Abajo,20,20,20,20,20,20


In [25]:
print('There are {} uniques categories.'.format(len(Santiago_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [26]:
# one hot encoding
Santiago_onehot = pd.get_dummies(Santiago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Santiago_onehot['Neighborhood'] = Santiago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Santiago_onehot.columns[-1]] + list(Santiago_onehot.columns[:-1])
Santiago_onehot = Santiago_onehot[fixed_columns]

Santiago_onehot.head()

,Zoo Exhibit,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Veterinarian,Video Game Store,Video Store,Vineyard,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
Santiago_onehot.shape

(3663, 274)

In [28]:
Santiago_grouped = Santiago_onehot.groupby('Neighborhood').mean().reset_index()
Santiago_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,...,Veterinarian,Video Game Store,Video Store,Vineyard,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,Apoquindo,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.025000
1,Bajos de Mena,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.166667,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000
2,Barrancas,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000
3,Barrio Bellavista,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.00000,0.000000,0.000000
4,Barrio Brazil,0.0,0.010000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.02,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.010000
5,Carrascal,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000
6,Chacarilla de Macul,0.0,0.000000,0.000000,0.000000,0.000000,0.015152,0.00,0.015152,0.00,...,0.000000,0.015152,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000
7,Chacra Valparaiso,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.00000,0.000000,0.010000
8,Chicureo Abajo,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000
9,Conchali,0.0,0.000000,0.000000,0.000000,0.000000,0.023810,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.023810,0.00,0.00,0.02381,0.000000,0.000000


In [29]:
Santiago_grouped.shape

(78, 274)

In [30]:
num_top_venues = 5

for hood in Santiago_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Santiago_grouped[Santiago_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Apoquindo----
                 venue  freq
0              Stadium  0.08
1                Plaza  0.08
2  Arts & Crafts Store  0.08
3             Pharmacy  0.08
4           Restaurant  0.05


----Bajos de Mena----
           venue  freq
0    Bus Station  0.17
1           Food  0.17
2           Park  0.17
3  Hot Dog Joint  0.17
4   Veterinarian  0.17


----Barrancas----
                venue  freq
0  Chinese Restaurant  0.13
1         Bus Station  0.13
2                Park  0.09
3               Plaza  0.09
4       Grocery Store  0.09


----Barrio Bellavista----
                       venue  freq
0                      Hotel  0.08
1                 Restaurant  0.07
2                    Theater  0.07
3             Sandwich Place  0.05
4  South American Restaurant  0.04


----Barrio Brazil----
                 venue  freq
0          Coffee Shop  0.11
1                  Bar  0.05
2  Peruvian Restaurant  0.05
3                Plaza  0.05
4       Sandwich Place  0.04


----Carrascal----
  

           venue  freq
0         Bakery  0.11
1    Bus Station  0.11
2  Grocery Store  0.11
3   Liquor Store  0.11
4           Park  0.11


----Los Cerrillos----
               venue  freq
0  Food & Drink Shop  0.08
1      Grocery Store  0.08
2   Sushi Restaurant  0.04
3               Park  0.04
4                Gym  0.04


----Los Nogales----
                  venue  freq
0              Pharmacy  0.10
1  Fast Food Restaurant  0.10
2           Bus Station  0.05
3                 Plaza  0.05
4                   Bar  0.05


----Los Quillayes----
                    venue  freq
0                   Plaza  0.14
1             Flea Market  0.14
2      Chinese Restaurant  0.14
3  Furniture / Home Store  0.07
4                  Tunnel  0.07


----Macul----
                venue  freq
0  Chinese Restaurant  0.11
1    Sushi Restaurant  0.07
2        Soccer Field  0.06
3          Restaurant  0.06
4            Pharmacy  0.05


----Maipu----
                  venue  freq
0                   Bar  0.0

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Santiago_grouped['Neighborhood']

for ind in np.arange(Santiago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Santiago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Apoquindo,Plaza,Pharmacy,Arts & Crafts Store,Stadium,Pizza Place
1,Bajos de Mena,Veterinarian,Food,Farm,Hot Dog Joint,Bus Station
2,Barrancas,Bus Station,Chinese Restaurant,Plaza,Park,Grocery Store
3,Barrio Bellavista,Hotel,Restaurant,Theater,Sandwich Place,Park
4,Barrio Brazil,Coffee Shop,Plaza,Peruvian Restaurant,Bar,Pizza Place
5,Carrascal,Liquor Store,Pizza Place,Fast Food Restaurant,Bakery,Sandwich Place
6,Chacarilla de Macul,Soccer Field,Chinese Restaurant,Bakery,Park,Farmers Market
7,Chacra Valparaiso,Chinese Restaurant,Bakery,Pizza Place,Gym,Sushi Restaurant
8,Chicureo Abajo,Coffee Shop,Gym,Housing Development,Harbor / Marina,Golf Course
9,Conchali,Furniture / Home Store,Department Store,Pharmacy,Sushi Restaurant,Bakery


## Clustering

In [33]:
# set number of clusters
kclusters = 5

Santiago_grouped_clustering = Santiago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Santiago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 4, 0, 0, 4, 4, 0, 0, 0, 2, 4, 2, 0, 0, 4, 2, 4, 2, 4, 0, 0,
       4, 4, 0, 4, 0, 0, 4, 4, 4, 4, 0, 0, 0, 4, 4, 0, 4, 0, 0, 4, 0, 0,
       0, 4, 0, 4, 0, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 1, 4,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
Santiago_venues.drop_duplicates(subset='Neighborhood',inplace=True)

In [35]:
Santiago_merged = Santiago_venues
# add clustering labels
Santiago_merged['Cluster Labels'] = kmeans.labels_

# merge Santiago_grouped with Santiago_data to add latitude/longitude for each neighborhood
Santiago_merged = Santiago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Santiago_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Apoquindo,-33.400000,-70.533333,Estadio San Jorge,-33.391910,-70.536040,Convention Center,0,Plaza,Pharmacy,Arts & Crafts Store,Stadium,Pizza Place
40,Bajos de Mena,-33.633333,-70.616667,Parque Juan Pablo II (ex-Cañamera),-33.624352,-70.612672,Park,2,Veterinarian,Food,Farm,Hot Dog Joint,Bus Station
46,Barrancas,-33.450000,-70.766667,pizza colombiana,-33.441901,-70.773167,Pizza Place,4,Bus Station,Chinese Restaurant,Plaza,Park,Grocery Store
69,Barrio Bellavista,-33.432778,-70.635833,Patio Bellavista,-33.434038,-70.635073,Food Court,0,Hotel,Restaurant,Theater,Sandwich Place,Park
169,Barrio Brazil,-33.438333,-70.666389,Emporio Vegetal,-33.439083,-70.668630,Organic Grocery,0,Coffee Shop,Plaza,Peruvian Restaurant,Bar,Pizza Place


In [36]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Santiago_merged['Neighborhood Latitude'], Santiago_merged['Neighborhood Longitude'], Santiago_merged['Neighborhood'], Santiago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
Santiago_merged.loc[Santiago_merged['Cluster Labels'] == 0]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Apoquindo,-33.400000,-70.533333,Estadio San Jorge,-33.391910,-70.536040,Convention Center,0,Plaza,Pharmacy,Arts & Crafts Store,Stadium,Pizza Place
69,Barrio Bellavista,-33.432778,-70.635833,Patio Bellavista,-33.434038,-70.635073,Food Court,0,Hotel,Restaurant,Theater,Sandwich Place,Park
169,Barrio Brazil,-33.438333,-70.666389,Emporio Vegetal,-33.439083,-70.668630,Organic Grocery,0,Coffee Shop,Plaza,Peruvian Restaurant,Bar,Pizza Place
356,Chacra Valparaiso,-33.466667,-70.600000,Piscina Olimpica Estadio Nacional,-33.462810,-70.606933,Gym Pool,0,Chinese Restaurant,Bakery,Pizza Place,Gym,Sushi Restaurant
456,Chicureo Abajo,-33.283333,-70.633333,Club De Golf Hacienda Chicureo,-33.291390,-70.634119,Golf Course,0,Coffee Shop,Gym,Housing Development,Harbor / Marina,Golf Course
476,Conchali,-33.378056,-70.674722,Tia Rosa,-33.375869,-70.675849,Fast Food Restaurant,0,Furniture / Home Store,Department Store,Pharmacy,Sushi Restaurant,Bakery
596,El Guanaco,-33.366667,-70.666667,Pasteleria Sacher,-33.365191,-70.670379,Bakery,0,Soccer Field,Sushi Restaurant,Furniture / Home Store,Pizza Place,Bakery
657,El Mirador,-33.416667,-70.683333,Parque Fluvial Renato Poblete,-33.424163,-70.678260,Water Park,0,Restaurant,Flower Shop,Sandwich Place,Food,Chinese Restaurant
837,Espejo,-33.533333,-70.716667,Centro Caja Los Andes,-33.532721,-70.716339,Campground,0,Moving Target,Diner,Soccer Field,Furniture / Home Store,Office
848,Independencia,-33.411667,-70.666389,Restaurant Nueva China,-33.407410,-70.661025,Chinese Restaurant,0,Plaza,Peruvian Restaurant,Restaurant,South American Restaurant,Chinese Restaurant


In [39]:
Santiago_merged.loc[Santiago_merged['Cluster Labels'] == 1]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2852,Rinconada de Chena,-33.566667,-70.733333,Taquería Lonquén,-33.559486,-70.734619,Taco Place,1,Soccer Field,Bar,Taco Place,Fish & Chips Shop,Falafel Restaurant


In [40]:
Santiago_merged.loc[Santiago_merged['Cluster Labels'] == 2]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
40,Bajos de Mena,-33.633333,-70.616667,Parque Juan Pablo II (ex-Cañamera),-33.624352,-70.612672,Park,2,Veterinarian,Food,Farm,Hot Dog Joint,Bus Station
518,El Almendro,-33.583333,-70.683333,Rosita Ríos,-33.578984,-70.676205,Food,2,Burger Joint,Pizza Place,Plaza,Pharmacy,Flea Market
560,El Bosque,-33.562778,-70.675556,Pibaz Gelato & Coffe,-33.553670,-70.676561,Ice Cream Shop,2,Flea Market,Bus Station,Ice Cream Shop,Liquor Store,Pharmacy
735,El Peumo,-33.433333,-70.750000,Persa Teniente Cruz,-33.436706,-70.741224,Flea Market,2,Plaza,Food Court,Flea Market,Burger Joint,Asian Restaurant
801,El Rosal,-33.566667,-70.666667,Centro Cívico El Bosque,-33.557193,-70.663588,Art Gallery,2,Food Truck,Flea Market,Art Gallery,Shop & Service,Grocery Store
2156,Los Quillayes,-33.566667,-70.616667,Xin Chung Restaurant,-33.555568,-70.623837,Chinese Restaurant,2,Plaza,Chinese Restaurant,Flea Market,Tunnel,Bus Station
2332,Mapuhue,-33.600000,-70.666667,Feria Libre Balmaceda,-33.595338,-70.669102,Flea Market,2,Argentinian Restaurant,Dog Run,Grocery Store,Bar,Flea Market
2848,Renca,-33.400000,-70.733333,Viveros de Renca,-33.403115,-70.734140,Garden,2,Market,Garden,Asian Restaurant,Grocery Store,Field


In [41]:
Santiago_merged.loc[Santiago_merged['Cluster Labels'] == 3]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3037,San Ramon,-33.45,-70.5,Sendero el Peumo - Parque Natural Aguas de Ramon,-33.437491,-70.496026,Mountain,3,Scenic Lookout,Mountain,Trail,Yoga Studio,Field


In [42]:
Santiago_merged.loc[Santiago_merged['Cluster Labels'] == 4]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
46,Barrancas,-33.450000,-70.766667,pizza colombiana,-33.441901,-70.773167,Pizza Place,4,Bus Station,Chinese Restaurant,Plaza,Park,Grocery Store
269,Carrascal,-33.416667,-70.716667,York Papas,-33.429486,-70.715264,Sandwich Place,4,Liquor Store,Pizza Place,Fast Food Restaurant,Bakery,Sandwich Place
290,Chacarilla de Macul,-33.500000,-70.583333,Donde El Pepe,-33.500174,-70.589045,Hot Dog Joint,4,Soccer Field,Chinese Restaurant,Bakery,Park,Farmers Market
532,El Blanqueado,-33.433333,-70.733333,Persa Teniente Cruz,-33.436706,-70.741224,Flea Market,4,Bakery,Nightclub,Sushi Restaurant,Plaza,Chinese Restaurant
679,El Peral,-33.583333,-70.566667,El Mesón de la Pizza,-33.588600,-70.567399,Pizza Place,4,Chinese Restaurant,Sushi Restaurant,Bus Station,Fast Food Restaurant,Grocery Store
744,El Retamo,-33.450000,-70.716667,Zen Sushi Ya,-33.444410,-70.724894,Sushi Restaurant,4,Bus Station,Bakery,Pharmacy,Sushi Restaurant,Nightclub
812,Errazuriz,-33.500000,-70.733333,D'mechadas,-33.498226,-70.739265,Sandwich Place,4,Chinese Restaurant,Park,Sandwich Place,Pharmacy,Grocery Store
913,La Africana,-33.433333,-70.850000,Upa! Ruta 68,-33.440287,-70.841529,Deli / Bodega,4,Airport,Fast Food Restaurant,Lake,Deli / Bodega,Golf Course
918,La Cisterna,-33.550000,-70.683333,Pibaz Gelato & Coffe,-33.553670,-70.676561,Ice Cream Shop,4,Sushi Restaurant,Pharmacy,Park,Food & Drink Shop,Airport
960,La Estrella,-33.550000,-70.600000,Gimnasio Gym club,-33.549531,-70.602778,Gym,4,Chinese Restaurant,Sushi Restaurant,Pharmacy,Bus Station,Soccer Field


## conclusions
Regarding the results found, it can be seen that in the north-east sector there is a cluster (red as well as the Peñalolen region) that presents the largest number of consumer stores, from which it can be inferred that the purchasing power of that region is greater, therefore that sector would be suitable to place the new Peumayen store.

Regarding the methodology used, Foursquare in Chile is in disuse, so the results are not 100% representative, for a better development you could use Google Maps API to improve the results of Venues.

At that same point, it was decided to increase the API radius to obtain a greater number of results and in turn, decrease the distance to the center of Santiago, since the page that was used to obtain the coordinates included surrounding cities that did not interest this analysis.